# 3D Reconstruction Tutorial: Data Processing, Surface Reconstruction, and Texture Mapping

Tiago Madeira, Lucal Dal'Col, Miguel Oliveira, and Paulo Dias

---

In this tutorial, we will walk through a complete 3D reconstruction pipeline.\
Press **<shift + enter>** to execute the current cell.

In [ ]:
# Importing libraries
import os
import sys
import time
import cv2
import pyfqmr
import pickle
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from tqdm import tqdm

# Custom modules
from texture_mapping import compute_texture, get_image_scores
from utils import get_cameras_data

---

Let's define the input arguments.

For logistical reasons, we have provided cached data in the **<./cache>** folder, instead of the original data source files.

In [ ]:
args = {
    'input_type': 'e57',
    'input_path': './024VARLab_web3d.e57',
    'cache_path': './cache',
    'output_path': './output',
    'voxel_size': 0.01,
    'max_decimation_error': 0.0001,
    'image_selection_algorithm': 'score',
}

    -type', '--input_type':
                  Type of input files.
                      'e57' -  E57 file containing laser scan, RGB camera captures and sensors parameters.
                      'conf' - Config file containing path to RGB and depth images along with camera parameters.

    -i', '--input_path':
                  Path to the input file.
                  
    -cache, --cache_path:
                  Path for cache directory.

    -o , --output_path:
                  Path to the output directory.
                  
    -voxel, --voxel_size:    (default: 0.008)
                  Size for finest level octree cells in meters.
                  
    -mde, --max_decimation_error:    (default: 0.00001)
                  Acceptable error for Fast Quadric Mesh Simplification.
                  
    -isa, --image_selection_algorithm:    (default: area)
                  Algorithm for image selection in texture mapping.
                      'random'  - Random image selection.
                      'area'  - Image with the largest projection available for triangle.
                      'score' - Image with the highest score (preserve continuity).

---

Load the data to memory.

Cached data is used to speed up the tutorial, but all the loading functionality is provided in utils.py.

In [ ]:
# Extract filename from the input path
file_out_name = os.path.basename(args['input_path']).split('.')[0]
print(file_out_name, 'dataset')

# Load the camera data
if args['cache_path'] and os.path.isfile(args['cache_path'] + '/camera_data.pickle'):
    with open(args['cache_path'] + '/camera_data.pickle', 'rb') as handle:
        cameras_data = pickle.load(handle)
else:
    cameras_data = get_cameras_data(args['input_path'], args['input_type'])


print('\nNumber of captures = ', len(cameras_data.keys()))
print('\nData contained in each capture: ')
print(list(cameras_data[0].keys()))

---

Let's explore this data to see what it looks like.

In [ ]:
# Loop through each capture
for capture_id, capture_data in cameras_data.items():
    print(f"Visualizing capture {capture_id + 1} / {len(cameras_data.items())}")
    
    # Extract imageBGR and scan_depth_image from the capture
    imageBGR = capture_data.get('imageBGR')
    depth_image = np.copy(capture_data.get('scan_depth_image'))
    
    # Convert BGR to RGB for proper display with matplotlib
    imageRGB = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)

    # Plot imageBGR
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(imageRGB)
    plt.title(f'Capture {capture_id} - RGB Image')
    plt.axis('off')

    # Plot scan_depth_images
    plt.subplot(1, 2, 2)
    plt.imshow(depth_image, cmap='gray')
    plt.title(f'Capture {capture_id} - Depth Image')
    plt.axis('off')

    # Display the images
    plt.show()

---

We will now define a function to integrate our depth and colour into a common volumentric data structure.

The `create_tsdf_from_rgbd` function generates a Truncated Signed Distance Function (TSDF) volume from RGB-D images.

We loop over each camera's data and integrate its RGB-D image into the TSDF volume using the camera pose given by the intrinsic and extrinsic camera parameters.

The TSDF represents the 3D scene as a volumetric grid. Each voxel (a 3D pixel) in this grid holds information about the distance to the nearest surface, allowing for efficient fusion of multiple 3D scans.

The signed distance function 𝑆(𝑝) at point 𝑝 is:

𝑆(𝑝) > 0 (outside the object),\
𝑆(𝑝) < 0 (inside the object),\
𝑆(𝑝) = 0 (on the surface).

𝑆(𝑝) is constructed using captures from different perspectives, where each pixel’s depth value provides a point on a surface in 3D space.

In [ ]:
# Function to create a Truncated Signed Distance Function (TSDF) volume from RGB-D images
def create_tsdf_from_rgbd(cameras_data, voxel_size, visualize = False):
    # Initialize a Scalable TSDF Volume
    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length= voxel_size,
        sdf_trunc= 10 * voxel_size,
        color_type= o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

    print("\nIntegrating RGBD data in TSDF...")
    for i, camera_id in enumerate(tqdm(cameras_data)):
            
        camera = cameras_data[camera_id]
        image = o3d.geometry.Image(cv2.cvtColor(camera['imageBGR'], cv2.COLOR_BGR2RGB))
        
        # Create RGBD image from color and depth
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            image,
            o3d.geometry.Image(camera['scan_depth_image'] * 1000),
            depth_trunc=100.0,
            convert_rgb_to_intensity=False)

        # Get intrinsic and extrinsic camera parameters
        K = camera['intrinsics']
        width = camera['width']
        height = camera['height']
        o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(width, height, K[0, 0], K[1, 1], K[0, 2], K[1, 2])

        # Integrate RGBD image into TSDF volume
        volume.integrate(rgbd, o3d_intrinsics, np.linalg.inv(camera['extrinsics']))
        
        ############################################
        # Visualization
        if visualize:
            pcd = volume.extract_point_cloud()
            
            def create_cube(color, size):
                cube_mesh = o3d.geometry.TriangleMesh.create_box(width=size, height=size, depth=size)
                color = np.clip(np.asarray(color), 0, 1)
                cube_mesh.paint_uniform_color(color)
                return cube_mesh

            voxels = o3d.geometry.TriangleMesh()
            colors = np.asarray(pcd.colors)
            
            for i in range(len(pcd.points)):
                point = pcd.points[i]
                color = colors[i]
                cube = create_cube(color, voxel_size * 0.90)
                cube.translate(point)
                voxels += cube
                
            o3d.visualization.draw_geometries([voxels], window_name=f"TSDF representation - Capture {camera_id}")
        
        ############################################

    return volume

---

Let's visualize the integration of each scan

In [ ]:
# Create TSDF volume from RGB-D images with larger voxel size for visualization
create_tsdf_from_rgbd(cameras_data, voxel_size=0.1, visualize = True)

In [ ]:
args['voxel_size'] = 0.005

# Create TSDF volume from RGB-D images
tsdf = create_tsdf_from_rgbd(cameras_data, voxel_size=args['voxel_size'], visualize = False)

---

Mesh extraction from the continuous function. The Marching Cubes algorithm generates a mesh by identifying and connecting surfaces in the 3D volume that correspond to the boundary of objects.

In [ ]:
# Extract mesh from TSDF volume using Marching Cubes algorithm
print("\nExtracting mesh...")
mesh = tsdf.extract_triangle_mesh()
# mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([mesh], window_name="Mesh before decimation")
print("Visualizing the mesh before decimation...")

---

In [ ]:
# Smooth the mesh surface (used for noisy data)
print("\nSmoothing mesh...")
# mesh = mesh.filter_smooth_laplacian(number_of_iterations=25, lambda_filter=0.5)
mesh = mesh.filter_smooth_taubin(number_of_iterations=10, lambda_filter=0.5, mu=-0.53)

---
Mesh decimation is the process of simplifying the 3D mesh to reduce its complexity, while keeping the most important geometric features intact. This is crucial when working with high-resolution models, as it makes processing faster and reduces memory usage.

In this step, we define a maximum allowable error for decimation using the **max_decimation_error** parameter. A smaller error value results in a higher-fidelity mesh, while a larger value allows more aggressive simplification.

Here, we use Quadric Mesh Simplification to reduce the mesh complexity. This algorithm works by iteratively collapsing edges while minimizing geometric distortion.


**Mesh Decimation**: Simplifies the mesh to reduce its complexity while preserving important geometric features.

**Mesh Cleanup**: The mesh is cleaned to remove errors such as duplicate vertices, triangles, and degenerate shapes.

In [ ]:
args['max_decimation_error'] = 0.0001

# Decimate the mesh using Quadric Mesh Simplification
print("\nDecimating mesh...")
mesh_simplifier = pyfqmr.Simplify()
mesh_simplifier.setMesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles))
mesh_simplifier.simplify_mesh(target_count=0, update_rate=1, max_iterations=20, aggressiveness=0,
                              preserve_border=True, alpha=args['max_decimation_error'], K=0, verbose=True)
vertices, faces, face_normals = mesh_simplifier.getMesh()

# Recreate simplified mesh
final_mesh = o3d.geometry.TriangleMesh()
final_mesh.vertices = o3d.utility.Vector3dVector(vertices)
final_mesh.triangles = o3d.utility.Vector3iVector(faces)

# Clean up the mesh topology
print("\nCleaning up topology...")
final_mesh.remove_duplicated_triangles()
final_mesh.remove_degenerate_triangles()
final_mesh.remove_duplicated_vertices()
final_mesh.remove_non_manifold_edges()
final_mesh.remove_unreferenced_vertices()

o3d.visualization.draw_geometries([final_mesh], window_name="Mesh after decimation")
print("Visualizing the final mesh after decimation...")

---

Z-buffering helps determine the visibility of different triangles of the mesh from the perspective of each camera. The Z-buffer (or depth buffer) stores the distance from the camera to the nearest object for each pixel, which is crucial for deciding which parts of the mesh are visible when rendering textures.

In [ ]:
print("\nGenerating data for Z-buffering...")
for camera_idx in tqdm(cameras_data):
    camera = cameras_data[camera_idx]
    renderer = o3d.visualization.rendering.OffscreenRenderer(camera['width'], camera['height'])
    mtl = o3d.visualization.rendering.MaterialRecord()
    mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
    mtl.shader = "defaultUnlit"
    renderer.scene.add_geometry("textured_mesh", final_mesh, mtl)
    K = camera['intrinsics']
    cam_matrix = camera['extrinsics']
    o3d_intrinsics = o3d.camera.PinholeCameraIntrinsic(camera['width'], camera['height'],
                                                        K[0, 0], K[1, 1], K[0, 2], K[1, 2])
    renderer.setup_camera(o3d_intrinsics, np.linalg.inv(cam_matrix))

    depth_image = np.array(renderer.render_to_depth_image(z_in_view_space=True))

    camera['mesh_depth'] = depth_image

---

Texture mapping is the process of applying a 2D image (called a texture) onto a 3D surface (the mesh) to give it detailed appearance or color. 
Instead of relying on just the mesh's geometry and colour per vertex, texture mapping allows the mesh to display colors, patterns, or surface details based on an image.

In [ ]:
args['image_selection_algorithm'] = 'area'

print("\nCreating texture map...")
textured_mesh = compute_texture(final_mesh, cameras_data, args['image_selection_algorithm'], img_filter=[])

---

We export the textured 3D mesh into a format that includes an .obj file, a .mtl file, and the associated image files for textures.


By exporting the mesh in this format, the textured mesh can be opened and visualized in many 3D software packages (e.g., Blender, Maya), or used in real-time applications (e.g., game engines). The .obj format is a widely supported standard for storing 3D models.

In [ ]:
print("\nExporting mesh...")
o3d.io.write_triangle_mesh(args['output_path'] + '/' + file_out_name + '.obj', textured_mesh,
                            write_triangle_uvs=True, print_progress=True)

---

At this stage, you can interactively explore your mesh and its texture mapping using the Graphical User Interface (GUI) provided by the visualization tool. This allows you to analyze how textures are applied to the surface of the 3D model.

To start, go to the File menu and select Open... to load the textured mesh.

**<Ctrl + Left Click>** on any face of the mesh to toggle information about how the texture is mapped onto that specific part of the model. This will show details such as texture coordinates and mapping boundaries.

This step is useful for inspecting the accuracy of texture placement and for troubleshooting any issues with the mapping.

In [ ]:
!python visualize_texture_mapping.py